# Data Collection

**Data Loading**

In [7]:
!pip install gdown

In [8]:
!gdown --folder "https://drive.google.com/drive/folders/14_aiMxHbjvUUJWs6XuT5of1Qd_iPWs-y" -O ../dataset/bank_data
!unzip "../dataset/bank_data/case 1.zip" -d ../dataset/bank_data

Retrieving folder contents
Processing file 1CTAUr3-03k6XaSrN-ZQjocX9dQMcvI7A case 1.zip
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1CTAUr3-03k6XaSrN-ZQjocX9dQMcvI7A
To: /Users/altemir_1/Desktop/personalized_notifications/ai/dataset/bank_data/case 1.zip
100%|█████████████████████████████████████████| 513k/513k [00:00<00:00, 908kB/s]
Download completed
Archive:  ../dataset/bank_data/case 1.zip
   creating: ../dataset/bank_data/case 1
  inflating: ../dataset/bank_data/case 1/clients.csv  
  inflating: ../dataset/bank_data/case 1/client_10_transactions_3m.csv  
  inflating: ../dataset/bank_data/case 1/client_10_transfers_3m.csv  
  inflating: ../dataset/bank_data/case 1/client_11_transactions_3m.csv  
  inflating: ../dataset/bank_data/case 1/client_11_transfers_3m.csv  
  inflating: ../dataset/bank_data/case 1/client_12_transactions_3m.csv  
  inflating: ../dataset/bank_data/ca

OSError: [Errno 5] Input/output error

**Importing Dependencies**

In [29]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt 
import seaborn as sns

**Dependencies Set Up**

In [101]:
# Pandas set up
pd.set_option('display.max_columns', None)

# Matplotlib set up
plt.rcParams.update({
    "font.size": 14,          # default text size
    "axes.titlesize": 16,     # title size
    "axes.labelsize": 14,     # x and y labels
    "xtick.labelsize": 12,    # x tick labels
    "ytick.labelsize": 12,    # y tick labels
    "legend.fontsize": 12,    # legend
})

**Data Merging**

In [37]:
# Create dict for each type of data
transfers = []
transactions = []

# Saving all csv files in respective lists
transfers_files = glob.glob('../dataset/bank_data/case 1/*transfers*.csv')
transactions_files = glob.glob('../dataset/bank_data/case 1/*transactions*.csv')

# Looping over the files and appending to the lists
for file in transfers_files:
    df = pd.read_csv(file)
    transfers.append(df)

for file in transactions_files:
    df = pd.read_csv(file)
    transactions.append(df)

# Merging all dataframes in the lists into single dataframes    
transfers_df = pd.concat(transfers, ignore_index=True)
transactions_df = pd.concat(transactions, ignore_index=True) 
clients_df = pd.read_csv('../dataset/bank_data/case 1/clients.csv')

# Data Preparation

**Helper functions**

In [110]:
# Calculate amount spend average in 3 month for categorical feautures
def calculate_average_spend(transactions, column):
    # Group by client_id and category, then sum the amount spent
    grouped = transactions.groupby(['client_code', column])['amount'].sum().unstack(fill_value=0)
    
    # Calculate the average spent per category over 3 months
    avg_spent = grouped / 3
    
    # Reset index to turn the client_id back into a column
    avg_spent.reset_index(inplace=True)
    
    return avg_spent

# Retrieve product and status from transactions
def get_product(transactions):
    # Select relevant columns and drop duplicates to get unique client_code, product, and status combinations
    product= transactions[['client_code', 'product']].drop_duplicates()
    
    return product

def convert_currency(transactions, exchange_rate=634):
    # Convert amounts in EUR to KZT using the provided exchange rate
    transactions.loc[transactions['currency'] == 'EUR', 'amount'] *= exchange_rate
    transactions.loc[transactions['currency'] == 'EUR', 'currency'] = 'KZT'
    
    return transactions

In [111]:
# Convert currency in both dataframes
transactions_df = convert_currency(transactions_df)
transfers_df = convert_currency(transfers_df)   

# Calculate spend for each category
avg_spend_category = calculate_average_spend(transactions_df, 'category')

# Calculate average amount per transaction type
avg_transfer_type = calculate_average_spend(transfers_df, 'type')

# Calculate average amount per transfer direction
avg_transfer_direction = calculate_average_spend(transfers_df, 'direction')

# Get product and status information
product_status = get_product(transactions_df)

# Merging all dataframes into a single dataframe
merged_df = clients_df.merge(avg_spend_category, on='client_code', how='left') \
                      .merge(avg_transfer_type, on='client_code', how='left', suffixes=('', '_transfer')) \
                      .merge(avg_transfer_direction, on='client_code', how='left', suffixes=('', '_direction')) \
                      .merge(product_status, on='client_code', how='left')

merged_df.fillna(0, inplace=True)

# Benefits Calculation

In [129]:
import pandas as pd
import numpy as np

def build_selection_features(df: pd.DataFrame, safety_buffer_kzt: int = 100_000) -> pd.DataFrame:
    """
    From your aggregated 3-month-avg client dataframe, compute ONLY the
    features needed to select the best financial product per client.

    Output columns (per client):
      - client_code
      - avg_monthly_balance_KZT
      - total_spend
      - travel_spend
      - online_services_spend
      - premium_boosted_spend
      - top3_spend
      - fx_volume
      - non_kzt_spend_approx
      - net_cashflow
      - eligible_balance
      - deposit_activity
      - debt_like
      - invest_activity
      - gold_activity
    """

    # Ensure numeric on required inputs (create zeros if missing)
    base_num = [
        'avg_monthly_balance_KZT', 'АЗС', 'Авто', 'Едим дома', 'Играем дома',
        'Кафе и рестораны', 'Кино', 'Косметика и Парфюмерия', 'Мебель',
        'Одежда и обувь', 'Отели', 'Подарки', 'Продукты питания', 'Путешествия',
        'Развлечения', 'Ремонт дома', 'Смотрим дома', 'Спорт', 'Такси',
        'Ювелирные украшения',
        'atm_withdrawal', 'card_out', 'p2p_out',
        'salary_in', 'stipend_in', 'family_in', 'refund_in', 'cashback_in', 'invest_in',
        'loan_payment_out', 'cc_repayment_out', 'installment_payment_out',
        'deposit_topup_out', 'fx_buy', 'fx_sell', 'gold_buy_out', 'gold_sell_in',
        'in', 'out'
    ]
    for col in base_num:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        else:
            df[col] = 0.0

    # Spending categories used for card benefits
    spend_cols = [
        'АЗС','Авто','Едим дома','Играем дома','Кафе и рестораны','Кино',
        'Косметика и Парфюмерия','Мебель','Одежда и обувь','Отели','Подарки',
        'Продукты питания','Путешествия','Развлечения','Ремонт дома',
        'Смотрим дома','Спорт','Такси','Ювелирные украшения'
    ]

    # Totals / grouped spends
    total_spend = df[spend_cols].sum(axis=1).fillna(0)

    travel_spend = (df['Такси'] + df['Отели'] + df['Путешествия']).fillna(0)

    online_services_spend = (
        df['Смотрим дома'] + df['Играем дома'] + df['Кино'] + df['Едим дома']
    ).fillna(0)

    premium_boosted_spend = (
        df['Ювелирные украшения'] + df['Косметика и Парфюмерия'] + df['Кафе и рестораны']
    ).fillna(0)

    # Top-3 categories by spend (only the sum is needed for product scoring)
    def _top3_sum(row):
        s = row[spend_cols].fillna(0)
        return float(s.sort_values(ascending=False).head(3).sum())
    top3_spend = df.apply(_top3_sum, axis=1)

    # FX / currency
    fx_volume = (df['fx_buy'] + df['fx_sell']).fillna(0)

    # Cashflow & debt signals
    income_salary = (
        df['salary_in'] + df['stipend_in'] + df['family_in'] +
        df['refund_in'] + df['cashback_in'] + df['invest_in']
    ).fillna(0)

    cash_outflow = (df['atm_withdrawal'] + df['p2p_out'] + df['card_out']).fillna(0)
    debt_like = (df['loan_payment_out'] + df['cc_repayment_out'] + df['installment_payment_out']).fillna(0)

    net_cashflow = (income_salary - (total_spend + cash_outflow + debt_like)).fillna(0)

    # Deposits & other signals
    deposit_activity = df['deposit_topup_out'].fillna(0)

    invest_activity = (df['gold_buy_out']*0)  # placeholder to keep shape if invest cols absent
    if 'invest_in' in df.columns or 'invest_out' in df.columns:
        invest_activity = (df.get('invest_in', 0).fillna(0) + df.get('invest_out', 0).fillna(0))
    gold_activity = (df['gold_buy_out'] + df['gold_sell_in']).fillna(0)

    # Assemble minimal feature frame
    out_cols = pd.DataFrame({
        'client_code': df['client_code'] if 'client_code' in df.columns else np.arange(len(df)),
        'avg_monthly_balance_KZT': df['avg_monthly_balance_KZT'].fillna(0),
        'total_spend': total_spend,
        'travel_spend': travel_spend,
        'online_services_spend': online_services_spend,
        'premium_boosted_spend': premium_boosted_spend,
        'top3_spend': top3_spend,
        'fx_volume': fx_volume,
        'net_cashflow': net_cashflow,
        'deposit_activity': deposit_activity,
        'debt_like': debt_like,
        'invest_activity': invest_activity,
        'gold_activity': gold_activity
    })

    return out_cols

In [130]:
final_df = build_selection_features(merged_df, safety_buffer_kzt=100_000)

/var/folders/vx/qnjn5crx6n17qty1n6m8kf4w0000gn/T/ipykernel_38762/1932019125.py:69: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  s = row[spend_cols].fillna(0)
/var/folders/vx/qnjn5crx6n17qty1n6m8kf4w0000gn/T/ipykernel_38762/1932019125.py:69: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  s = row[spend_cols].fillna(0)
/var/folders/vx/qnjn5crx6n17qty1n6m8kf4w0000gn/T/ipykernel_38762/1932019125.py:69: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=Fal

,client_code,avg_monthly_balance_KZT,total_spend,travel_spend,online_services_spend,premium_boosted_spend,top3_spend,fx_volume,net_cashflow,deposit_activity,debt_like,invest_activity,gold_activity
0,1,92643,8.756381e+05,214013.633333,219587.110000,1.729192e+05,4.798195e+05,0.000000e+00,-2.385929e+06,0.000000,1.693368e+05,0.000000e+00,0.000000e+00
1,2,1577073,8.744241e+05,145964.673333,207453.876667,1.714567e+05,5.296388e+05,0.000000e+00,-2.473845e+06,0.000000,1.685445e+05,0.000000e+00,0.000000e+00
2,3,63116,7.946233e+05,273591.416667,190710.933333,1.355907e+05,4.516705e+05,0.000000e+00,-1.678318e+06,0.000000,1.866081e+05,0.000000e+00,0.000000e+00
3,4,83351,8.469978e+05,204708.496667,196531.226667,1.742984e+05,4.721151e+05,0.000000e+00,-2.216535e+06,0.000000,1.771722e+05,0.000000e+00,0.000000e+00
4,5,1336536,9.012237e+05,213650.060000,223623.463333,1.413203e+05,4.729691e+05,0.000000e+00,-2.328407e+06,0.000000,1.826814e+05,0.000000e+00,0.000000e+00
5,6,131929,8.900856e+05,195838.356667,194450.180000,1.874670e+05,5.515705e+05,0.000000e+00,-2.394871e+06,0.000000,1.604846e+05,0.000000e+00,0.000000e+00
6,7,4040997,9.006073e+05,78368.070000,210567.630000,2.907908e+05,5.464896e+05,0.000000e+00,-3.018223e+06,0.000000,3.291624e+05,0.000000e+00,0.000000e+00
7,8,1058403,1.071709e+06,78509.783333,206995.746667,4.242892e+05,7.090593e+05,0.000000e+00,-3.124604e+06,0.000000,3.140596e+05,0.000000e+00,0.000000e+00
8,9,3084180,9.412963e+05,74426.613333,197628.423333,3.197684e+05,6.150759e+05,0.000000e+00,-3.116972e+06,0.000000,3.116346e+05,0.000000e+00,0.000000e+00
9,10,1277325,9.731782e+05,66312.016667,180037.280000,3.386275e+05,6.496894e+05,0.000000e+00,-3.069899e+06,0.000000,2.631481e+05,0.000000e+00,0.000000e+00
